In [1]:
import sys, os, glob, datetime, cftime,pickle
import xarray as xr
import numpy as np
import pandas as pd
from scipy import float64
from dill.source import getsource
import regionmask

sys.path.append('../')

from _decomp import *
from _data_minimal import *
from _print_and_plot import *


from windspharm.xarray import VectorWind
from windspharm.examples import example_data_path

%load_ext autoreload 
%autoreload 2

In [2]:
sorted(glob.glob('/climca/data/CESM2-ETH/*/U_day_*'))

['/climca/data/CESM2-ETH/b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1300.linear-weak/U_day_b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1300.linear-weak.nc',
 '/climca/data/CESM2-ETH/b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1400.linear-weak/U_day_b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1400.linear-weak.nc',
 '/climca/data/CESM2-ETH/b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1500.linear-weak/U_day_b.e212.B1850cmip6.f09_g17.001.nudge-1850-2100-SSP370.1500.linear-weak.nc',
 '/climca/data/CESM2-ETH/b.e212.BHISTcmip6.f09_g17.1300/U_day_b.e212.BHISTcmip6.f09_g17.1300.nc',
 '/climca/data/CESM2-ETH/b.e212.BHISTcmip6.f09_g17.1400/U_day_b.e212.BHISTcmip6.f09_g17.1400.nc',
 '/climca/data/CESM2-ETH/b.e212.BHISTcmip6.f09_g17.1500/U_day_b.e212.BHISTcmip6.f09_g17.1500.nc',
 '/climca/data/CESM2-ETH/b.e212.BSSP370cmip6.f09_g17.1300/U_day_b.e212.BSSP370cmip6.f09_g17.1300.nc',
 '/climca/data/CESM2-ETH/b.e212.BSSP370cmip6.f09_g17.1400/U_day_b.e212.BSSP3

In [5]:
for fl in sorted(glob.glob('/climca/data/CESM2-ETH/*/U_day_*'))[::-1]:
    print(fl)
    out_file = f"{fl.replace('U_day_','XXX_day_').replace('/climca/data/CESM2-ETH','/climca/people/ppfleiderer/CESM2-ETH_postprocessed')}"
    out_path = '/'.join(out_file.split('/')[:-1])
    if os.path.isdir(out_path) == False:
        os.system(f"mkdir -p {out_path}")
    cdo.sellevel(
        '50000',
        input=f"{fl}",
        output=out_file.replace('XXX','U_500hPa'),
        force =  False,
        options = "-P 4"
    )
    cdo.sellevel(
        '50000',
        input=f"{fl.replace('U_day_','V_day_')}",
        output=out_file.replace('XXX','V_500hPa'),
        force =  False,
        options = "-P 4"
    )
    cdo.dv2ps(
        input=f"-uv2dv -remapbil,F180 -merge {out_file.replace('XXX','U_500hPa')} {out_file.replace('XXX','V_500hPa')}",
        output=out_file.replace('XXX','streamVelopot_spherical_500hPa'),
        force =  False,
        options = "-P 4"
    )
    cdo.remapbil(
        fl,
        input=f"-sp2gp {out_file.replace('XXX','streamVelopot_spherical_500hPa')}",
        output=out_file.replace('XXX','streamVelopot_500hPa'),
        force =  False,
        options = "-P 4"
    )


/climca/data/CESM2-ETH/b.e212.BSSP370cmip6.f09_g17.1500/U_day_b.e212.BSSP370cmip6.f09_g17.1500.nc
/climca/data/CESM2-ETH/b.e212.BSSP370cmip6.f09_g17.1400/U_day_b.e212.BSSP370cmip6.f09_g17.1400.nc
